# Lab assignment: unemployment rates

In this assignment we will estimate unemployment rates in Spain, using *pandas* for reading information, *scikit-learn* for training estimators, and *geopandas* and *folium* for visualizing results.

## 0. Guidelines

Throughout this notebook you will find empty cells that you will need to fill with your own code. Follow the instructions in the notebook and pay special attention to the following symbols.

<table align="center">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">You will need to solve a question by writing your own code or answer in the cell immediately below or in a different file, as instructed.</td></tr>
 <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">This is a hint or useful observation that can help you solve a question. You should pay attention to these hints to better understand questions.</td></tr>
 <tr><td width="80"><img src="img/pro.png" style="width:auto;height:auto"></td><td style="text-align:left">This is an advanced and voluntary exercise that can help you gain a deeper knowledge into the topic. Good luck!</td></tr>
</table>

During the assignment you will make use of several Python packages that might not be installed in your machine. It is strongly recommended that you use the environment file *environment.yml* and follow the instructions of the following <a href="https://github.com/jorloplaz/teaching_material/tree/master/SVM">link</a>.

If you need any help on the usage of a Python function you can place the writing cursor over its name and press Caps+Shift to produce a pop-out with related documentation. This will only work inside code cells.

Make sure the following cell executes correctly, as it imports all you will need:

In [ ]:
import pandas as pd
import numpy as np
import branca.colormap as cm
import utils
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR, SVR
from sklearn.metrics import r2_score
from math import ceil
%matplotlib inline

## 1. Load and understand data

To begin with, let us read some **numerical characteristics of Spanish municipalities**: 

In [ ]:
muns_population = utils.read_population('./data/municipios.csv')
print(len(muns_population))
muns_population.head()

As you can see, **for each municipality we have its position (latitude and longitude in degrees), its altitude (in meters) and its population (number of inhabitants)**. The resulting dataframe is **indexed in a hierarchical way, first by autonomous community, then by province, and finally by municipality name, for a total of 8112 municipalities**.

Our goal is to **predict the yearly unemployment rate for each municipality in 2017**. Let us read the unemployment data belonging to that year:

In [ ]:
unempl_2017 = utils.read_unemployment('./data/Paro_por_municipios_2017.csv')   # use specific load function from utils module
print(len(unempl_2017), len(unempl_2017.index.unique()))
unempl_2017.head()

These data are indexed in the same way as *muns_population* above. However, we are interested in yearly unemployment, but **we have monthly information** (actually, 97512 observations from 8126 municipalities). For example, taking the municipality of Abla:

In [ ]:
unempl_2017.xs(key='Abla', level=utils.MUN_FIELD, drop_level=False)      # all records whose mun is Abla, keeping index intact

Using grouping, it is fairly easy to obtain the **yearly means** for each municipality:

In [ ]:
unempl_2017_means = unempl_2017.groupby(unempl_2017.index.names)[[utils.UNEMPL_FIELD]].mean()   # group by all index levels, then take the mean
assert(len(unempl_2017_means) == len(unempl_2017) // 12)       # there should be just 1 row for each municipality
unempl_2017_means.head()

Now, in order to obtain our target values, we just need to transform these raw figures to unemployment rates, that is, we have to **divide unemployment by each municipality's population**:

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
 Create a <i>Y</i> matrix with the mean unemployment rates.
 </td></tr>
</table>

<table align="left">
<tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
    You should join the frames <i>unempl_2017_means</i> and <i>muns_population</i>. Then divide the mean unemployed people by the inhabitants. 
</td></tr>
</table>

<table align="left">
<tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Be careful while joining. There are some municipalities present in <i>unempl_2017_means</i> but missing in <i>muns_population</i> (8126 versus 8112), which can lead to <i>nan</i> values when dividing.
</td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

Make sure you did things correctly:

In [ ]:
assert(len(Y) <= min(len(unempl_2017_means), len(muns_population)))     # join cannot be larger than each of the joined frames
assert(isinstance(Y, pd.Series) or (isinstance(Y, pd.DataFrame) and len(Y.columns) == 1))     # a series or a frame with just 1 column

<table align="left">
 <tr><td width="80"><img src="img/pro.png" style="width:auto;height:auto"></td><td style="text-align:left">
Can you list the municipalities with known unemployment but unknown population? Conversely, are there any municipalities with known population but unknown unemployment?
 </td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

Great, so we already have Y, but we need patterns X to train a model. Which features should we use? Well, to begin with **we can use all features from muns\_population, since they are numerical**:

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
     Create an <i>X</i> matrix with the numerical features for each municipality. Considering that in <i>Y</i> we have unemployment rates (unemployed people divided by inhabitants), is it cheating if we use the number of inhabitants as a feature? Why/why not? 
 </td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

You should have obtained X and Y in the same order:

In [ ]:
assert(isinstance(X, pd.DataFrame)) 
assert(all(X.columns == muns_population.columns))
assert(all(X.index == Y.index))

## 2. Baseline models

Cool, we are ready to exploit Machine Learning. We just need to **split the data into training and set sets**. Let us fix the random seed and the ratio of data for training and test:

In [ ]:
RANDOM_STATE = 42   # fix seed
TEST_PERC = 0.3     # 30% for test, 70% for training

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
     Split <i>X</i> and <i>Y</i> into training and test sets, so that you obtain variables <i>X_train</i>, <i>Y_train</i>, <i>X_test</i> and <i>Y_test</i>. Use both constants <i>RANDOM_STATE</i> and <i>TEST_PERC</i>.
 </td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

Let us verify that data have been split correctly:

In [ ]:
assert(len(X_test) == ceil(TEST_PERC*len(X)) and len(Y_test) == len(X_test))       # test set size is 30% of total size
assert(len(X_train) == len(X)-len(X_test) and len(Y_train) == len(X_train))     # every pattern not in test should be in train

And create **colors to plot in a map the unemployment rate of each municipality**:

In [ ]:
cmap = cm.LinearColormap(colors=['green', 'yellow', 'orange', 'red', 'black'], 
                                 vmin=round(Y.min(), 2), vmax=round(Y.max(), 2), 
                                 caption='Unemployment rate')
cmap

The following code generates the map corresponding to the true values of patterns in the test set, saving this map as an HTML document:

In [ ]:
coms = utils.read_communities('./data/ComunidadesAutonomas_ETRS89_30N/')      # read communities
provs = utils.read_provinces('./data/Provincias_ETRS89_30N/')     # read provinces
_ = utils.generate_map(pd.concat([X_test, Y_test], axis=1),      # generate map, which needs X and Y together...
                       utils.LAT_FIELD, utils.LON_FIELD, Y_test.name if Y_test.name is not None else 0,   # columns with latitude, longitude and unemployment...
                       cmap, coms=coms, provs=provs, filename='./test_true.html')    # colors, communities, provinces and where to store the map

If you open the HTML with a browser, you will see the test municipalities plotted in the map. Ideally, **that is what we should predict**. Will an SVM be able to reproduce it?

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
     Train a <b>linear SVR</b> with the dataset created above, using cross-validation to find the most suitable values for its hyperparameters.
 </td></tr>
</table>

<table align="left">
<tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Make sure you normalize inputs before feeding them to the SVM. Use a <i>Pipeline</i> with a <i>StandardScaler</i>.
</td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

Let us try to **interpret what the resulting SVM is doing**:

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
Generate predictions for the test set with the best linear SVR you obtained. Do you observe overfitting? Plot these predictions in a map as we did above. How good/bad is it performing, visually speaking? Do you see any strange behaviour?
 </td></tr>
</table>

<table align="left">
<tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
    You just need to call <i>generate_map</i>. Do not read communities nor provinces again, cause they have been already loaded in memory.
</td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
Explain what the model obtained is doing. What is the intercept and why is it that? What is the importance of each feature? Which features have a positive (i.e., making unemployment bigger) or a negative (making it smaller) effect on unemployment? Does this agree with your intuition about the problem?
 </td></tr>
</table>

<table align="left">
<tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
    The intercept (<i>b</i> in our formulas) is stored in the <i>intercept_</i> attribute of the model, whereas the weight vector (<i>w</i> in our formulas) is kept in the <i>coef_</i> attribute.
</td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

Let us **check if we can do better with a non-linear SVM**.

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
     Train a <b>non-linear SVR with the RBF kernel</b>, tuning its hyperparameters with cross-validation.
 </td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
Create the map with the non-linear SVR predictions. Did things improve?
 </td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table align="left">
 <tr><td width="80"><img src="img/pro.png" style="width:auto;height:auto"></td><td style="text-align:left">
Try to explain the behaviour of the non-linear model obtained.
</table>

<table align="left">
<tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
It is much more difficult to interpret a non-linear SVM than a linear one. You can examine the support vectors, available in the attributes <i>support_</i>, <i>support_vectors_</i> and <i>dual_coef_</i>, to see which municipalities are considered as the most and least representative of each class. Another idea is to check and detect trends in municipalities unemployment being underestimated, being overestimated or being accurately predicted.
</td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

## 3. Refined models

### 3.1. Previous years (2006-2015)

So far we have predicted unemployment rates for 2017 based solely on municipality fixed characteristics (namely coordinates, altitude and population). Fortunately, we also have the unemployment figures for years 2006-2016, so that **we can also use unemployment in previous years to try to predict what will happen next**, in a time series fashion. Omitting for the moment 2016 (we will use it later):

In [ ]:
prev_years = range(2006, 2016)    # does not include 2016
prev_years

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
     Read the unemployment data from the <i>./data</i> folder for years 2006-2015 (both 2006 and 2015 included). You should concatenate the data for all years in a single dataframe called <i>unempl_2006_2015</i>.
 </td></tr>
</table>

<table align="left">
<tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
    The function <i>read_unemployment</i> from the <i>utils</i> module will help you to load the unemployment figures for a single year, as was done above for year 2017. Each year has a separate CSV file, so you should call <i>read_unemployment</i> once for each file. Then use pandas <a href="http://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html">concat</a> method to store everything in a single dataframe.
</td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

**For each municipality, we expect to have 120 observations** (10 years, 12 months each). For example, again taking Abla in Almería the last 20 are:

In [ ]:
unempl_2006_2015.xs(key='Abla', level=utils.MUN_FIELD, drop_level=False).tail(20)      # all records whose mun is Abla, keeping index intact

**Again we can take yearly means** for each year:

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
Calculate unemployment yearly means for each year 2006-2015 and each municipality, in a similar way that was done previously for year 2017.
 </td></tr>
</table>

<table align="left">
 <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
Since there are several years now, you should also group by year besides grouping by community, province and municipality.
 </td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

And **transform these raw means to rates**:

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
     Create an <i>X2</i> matrix with the mean unemployment rates for the different years. This matrix should have one row for each municipality and one column for each year.
 </td></tr>
</table>

<table align="left">
<tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
You have to do two things: 
<ol>
<li> Join with populations and divide.
<li> Transpose from column to row format. 
</ol> 
1) is basically done as for 2017 (joining on just the first 3 levels, omitting the year), while 2) is easily tackled with the <a href="http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot.html">pivot</a> method.
</td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

Verify that your matrix is correct:

In [ ]:
assert(all(X2.index == X.index) and (len(X2.columns) == len(prev_years)))      # same exact municipalities than before, one column for each year

**Split these new patterns**:

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
     Split <i>X2</i> into training and test sets, so that you obtain variables <i>X2_train</i> and <i>X2_test</i>.
 </td></tr>
</table>

<table align="left">
 <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
     Do not call <i>train_test_split</i> again, because <i>Y</i> is already split. Just index <i>X2</i> with the indices in <i>X_train</i> and <i>X_test</i>, because municipalities are the same.
 </td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

Training and test municipalities should be the same as before:

In [ ]:
assert(all(X2_train.index == X_train.index) and all(X2_train.columns == X2.columns) and     # same index, same columns for training
      all(X2_test.index == X_test.index) and all(X2_test.columns == X2_train.columns))     # and for testing

**Train and predict with a linear SVR with the new dataset**:

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
     Tune a <i>linear SVR</i>, predict with it, plot tits predictions and compare with the ones you got for the baseline linear SVR. Are these predictions better? By how much?
 </td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
Explain what this new linear model is doing. Which do you think are the most relevant features now? Verify whether your intuition is correct or not.
 </td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

**See if a non-linear SVR can improve further**:

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
     Tune a <b>non-linear SVR</b>, predict with it, plot its predictions and compare with the ones you got for the baseline non-linear SVR, and also with the ones you just obtained for the linear SVR. Is it worth now going from linear to non-linear?
 </td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table align="left">
 <tr><td width="80"><img src="img/pro.png" style="width:auto;height:auto"></td><td style="text-align:left">
Try to explain the behaviour of the new non-linear model obtained.
</table>

In [ ]:
####### INSERT YOUR CODE HERE

### 3.2 Previous years 2006-2015 + 2016

Time to **check whether introducing year 2016 helps or not**:

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
     Create an <i>X3</i> matrix with the same information as <i>X2</i> plus the unemployment mean rates for 2017 (i.e., an additional column). Split it into <i>X3_train</i> and <i>X3_test</i>.
 </td></tr>
</table>

<table align="left">
 <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
Do not process again years 2006-2015. You just have to read the information for 2017, process it analogously to what we did above and concatenate the resulting column with the ones of <i>X2</i>.
 </td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

Your matrices should have incorporated the additional column for 2017, and kept the rest with no changes:

In [ ]:
assert(all(X3_train.index == X2_train.index) and all((c in X3_train.columns) for c in X2_train.columns) 
       and len(X3_train.columns) == len(X2_train.columns)+1)     # same index, same columns plus the extra one
assert(all(X3_test.index == X2_test.index) and all(X3_test.columns == X3_train.columns))    # same index, same columns as for train

**Train the new linear model**:

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
     Tune a <b>linear SVR</b> and compare results with the ones you obtained for 2006-2015. Is 2016 helping? What happens now with feature relevance compared to the previous one?
 </td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

And **finally a non-linear model**:

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
     Tune a <b>non-linear SVR</b> and compare results with the ones you obtained for 2006-2015. Any better?
 </td></tr>
</table>

In [ ]:
####### INSERT YOUR CODE HERE

## 4. Summary

<table align="left">
 <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
Summarise the results obtained. There should be 6 models to compare:
<ol>
<li> Baseline with municipality characteristics (linear and non-linear).
<li> Unemployment history 2006-2015 (linear and non-linear).
<li> Unemployment history 2006-2016 (linear and non-linear).
</ol>
Which model would you choose if you had to go into production? Why?
 </td></tr>
</table>

## 5. Bonus round

<table align="left">
 <tr><td width="80"><img src="img/pro.png" style="width:auto;height:auto"></td><td style="text-align:left">
Explore further how we can generate even better predictions. The following are possible lines:
<ul>
<li>Combining variables from municipalities and past unemployment rates.</li>
<li>Using the original monthly data instead of yearly means.</li>
<li>Downloading and using additional data (economical, meteorological...) from other sources.</li>
</ul>
Feel free to edit this notebook from now on explaining your approach.
 </td></tr>
</table>

<table align="left">
 <tr><td width="80"><img src="img/pro.png" style="width:auto;height:auto"></td><td style="text-align:left">
Imagine that instead of predicting the mean unemployment rate for 2017 you were asked to predict monthly unemployment rates for 2017. In other words, you would like to predict for January 2017, then for February 2017, and so on till December 2017.

Explain how you would tackle this problem with the data available. Extra points if you implement your proposal and show how it works in practice. Feel free to edit this notebook from now on explaining your approach.
 </td></tr>
</table>